In [1]:
# import library
import pickle, os
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import copy
from scipy.stats.mstats import winsorize

In [ ]:
pip install sirius
데이터 다운로더 자체를 내장시키는 방법도 가능
pip install Moggle_datadownloader  (셀레니움으로 KRX데이터를 다운로드 / fundamental data 다운로드)
config 파일 생성 ->  directory

sirius를 가지고 API 같은 느낌이 나도록? : REST API로 구현이 되어 있는데 이걸 아예 sirius 안에 내장시켜도 좋을듯!

홈페이지를 재정비

In [3]:
equity = get_item_for_period('ifrs_Equity', '20131231', '20191231', term = 'Y')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [2]:
# Data Download
with open('FS_손익계산서보정본', 'rb') as f:
    data = pickle.load(f)

with open('Final_FS_ind', 'rb') as f:
    ind = pickle.load(f)

# 특정 자산만 뽑아서 csv로 정리하기
def get_item(concept_id):
    item = []
    item_ind = []
    for i in range(len(data)):
        if type(data[i][0]) == type(data[0][0]):
            item.append(data[i][0][data[i][0][data[i][0].columns[0]] == concept_id])
            if len(data[i][0][data[i][0][data[i][0].columns[0]] == concept_id]) > 0:
                item_ind.append(ind[i])
    df_item = pd.concat(item)
    date = list(filter(lambda x: x[1] == ('연결재무제표',), df_item.columns))
    
    date_clean = list(map(lambda x: x[0], df_item[date].columns))
    df_item[date].columns = date_clean
    df_item2 = df_item[date]
    df_item2.columns = date_clean
    date_shifted = list(map(lambda x: shift_q(x), date_clean))
    df_item3 = df_item2
    df_item3.columns = date_shifted
    df_item4 = df_item3.groupby(level=0, axis=1).last()
    df_item3 = df_item4
    date_shifted2 = list(map(lambda x: shift_d(x), df_item4.columns))
    df_item3.columns = date_shifted2
    df_item5 = df_item3.groupby(level=0, axis=1).last()
    corp_code = list(map(lambda x:x[2], item_ind))
    df_item5['corp_code'] = corp_code
    df_item5 = df_item5.set_index('corp_code')
    return df_item5

def shift_q(date):
        if date[4:6]=='01' or date[4:6]=='02':
            date = date[:4] + '0331'
        elif date[4:6]=='04' or date[4:6]=='05':
            date = date[:4] + '0630'
        elif date[4:6]=='07' or date[4:6]=='08':
            date = date[:4] + '0930'
        elif date[4:6]=='10' or date[4:6]=='11':
            date = date[:4] + '1231'
        return date

def shift_d(date):
    # 케이스 두개 밖에 안되니까 그냥 수동으로 따져주자.
    if date[6:]=='01':
        date = date[:6] + '31'
    elif date[6:]=='29':
        date = date[:6] + '30'
    return date

def get_item_on_date(item, date):
    return get_item(item)[date]

def get_item_for_period(item, date_from, date_to, term = 'Y'):
    # term은 Y, H, Q의 세 가지 옵션으로, 연도별, 반기별, 분기별 데이터를 얻어올 수 있도록 하는 옵션이다.
    data = []
    item_data = get_item(item)
    if term == 'Y':
        num = int(date_to[:4]) - int(date_from[:4]) +1
        date_working = date_from
        for i in range(num):
            data.append(item_data[date_working])
            date_working = date_working[:2] + str(int(date_working[2:4]) + 1) + date_working[4:]
            
    elif term == 'H':
        num = int((int(date_to[:4]) - int(date_from[:4])) * 2 + (int(date_to[4:6]) - int(date_from[4:6]))/6 + 1)
        date_working = date_from
        for i in range(num):
            try:
                data.append(item_data[date_working])
            except:
                try:
                    date_working = date_working[:6] + str(61 - int(date_working[6:]))
                    data.append(item_data[date_working])
                except KeyError:
                    print('해당 날짜의 데이터가 없습니다:', date_working)
            if int(date_working[4:6])+6 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0'+ str(int(date_working[4:6])-6)+'31'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 6).zfill(2) + '31'
                
    else:
        num = int((int(date_to[:4]) - int(date_from[:4])) * 4 + (int(date_to[4:6]) - int(date_from[4:6]))/3 + 1)
        date_working = date_from
        for i in range(num):
            try:
                data.append(item_data[date_working])
            except:
                try:
                    date_working = date_working[:6] + str(61 - int(date_working[6:]))
                    data.append(item_data[date_working])
                except KeyError:
                    print('해당 날짜의 데이터가 없습니다:', date_working)
            if int(date_working[4:6])+3 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0331'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 3).zfill(2) + '31'
                
    data = pd.concat(data, axis = 1)
    return data

os.chdir('C:\\Users\\lyoo9\\_System Trading\\Backend\\KRX_price')
def get_market_data_on_date(date, print_error = True):
    # 에러 메시지를 출력하고 싶지 않으면 print_error를 False로 설정
    try:
        data = pd.read_csv(date[2:4] + '_' + str(int(date[4:6])) + '_' + str(int(date[6:8])) + '.csv', encoding = 'cp949')
        data['종목코드'] = data['종목코드'].map(lambda x:str(x).zfill(6) if x != None else None)
        if data.isna()['종가'].sum() == len(data):
            raise FileNotFoundError
    except FileNotFoundError:
        if print_error:
            print('해당 날짜의 데이터가 존재하지 않습니다:', date)
        data = None
    return data

def get_market_cap_on_date(date, get_code = False):
    if get_code:
        data = get_market_data_on_date(date)[['종목코드','시장구분','시가총액']].set_index('종목코드')
        data.columns = [['시장구분','Market_Cap']]
    else:
        data = get_market_data_on_date(date)[['종목코드','시가총액']].set_index('종목코드')
        data.columns = [['Market_Cap']]
    data.index.rename('corp_code', inplace = True)
    return data

def last_market_date(date):
    # date의 기본형식은 'YYYYMM' : ex)'201212'. -> 'YYYYMMDD'도 가능하도록 코드 변경했음
    get_data = False
    day = 31
    while not get_data:
        if type(get_market_data_on_date(date[:6] + str(day), print_error = False)) != type(None):
            market_date = date[:6] + str(day)
            get_data = True
        else:
            day -=1
    return market_date

def find_next_market_date(date, today_opt = False):
    get_data = False
    if today_opt:
        market_date = pd.Timestamp(date)
    else:
        market_date = pd.Timestamp(date) + pd.Timedelta(days=1)
    while not get_data:
        if type(get_market_data_on_date(market_date.strftime('%Y%m%d'), print_error = False)) != type(None):
            get_data = True
        else:
            market_date += pd.Timedelta(days=1)
    
    return market_date.strftime('%Y%m%d')

def get_market_cap_for_period(date_from, date_to, term = 'Y', ind_match = True):
    # ind_match는 fundamental과 합치기 편하도록 실제 날짜가 아니라 월말 날짜를 표시하도록 하는 옵션
    data = []
    ind = []
    if term == 'Y':
        num = int(date_to[:4]) - int(date_from[:4]) +1
        date_working = date_from
        for i in range(num):
            data.append(get_market_cap_on_date(last_market_date(date_working), get_code = True))
            if ind_match:
                ind.append(date_working)
            else:
                ind.append(last_market_date(date_working))
            date_working = date_working[:2] + str(int(date_working[2:4]) + 1) + date_working[4:]
            
    elif term == 'H':
        num = int((int(date_to[:4]) - int(date_from[:4])) * 2 + (int(date_to[4:6]) - int(date_from[4:6]))/6 + 1)
        date_working = date_from
        for i in range(num):
            data.append(get_market_cap_on_date(last_market_date(date_working), get_code = True))
            if ind_match:
                ind.append(date_working)
            else:
                ind.append(last_market_date(date_working))
            if int(date_working[4:6])+6 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0'+ str(int(date_working[4:6])-6)+'31'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 6).zfill(2) + '31'
                
    else:
        num = int((int(date_to[:4]) - int(date_from[:4])) * 4 + (int(date_to[4:6]) - int(date_from[4:6]))/3 + 1)
        date_working = date_from
        for i in range(num):
            data.append(get_market_cap_on_date(last_market_date(date_working), get_code = True))
            if ind_match:
                ind.append(date_working)
            else:
                ind.append(last_market_date(date_working))
            if int(date_working[4:6])+3 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0331'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 3).zfill(2) + '31'
    cap_data = [x.iloc[:,1] for x in data]
    cat_data = [x.iloc[:,0] for x in data]
    cap_data = pd.concat(cap_data, axis =1)
    cap_data.columns = ind
    cat_data = pd.concat(cat_data, axis =1)
    cat_data.columns = ind
    return cap_data, cat_data

def get_market_category(data):
    market_cat = []
    for i in range(len(data.columns)):
        market_cat.append(get_market_cap_on_date(last_market_date(data.columns[i]), get_code = True).iloc[:,0])
    market_cat = pd.concat(market_cat, axis=1)
    market_cat.columns = data.columns
    return market_cat

In [25]:
see = get_item_for_period('ifrs_Equity', '20131231', '20201231') / get_market_cap_for_period('20131231', '20201231')[0]

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [28]:
get_market_category(equity)

,20131231,20140630,20141231,20150630,20151231,20160630,20161231,20170630,20171231,20180630,20181231,20190630,20191231,20200630,20201231
060310,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ
013340,KOSDAQ,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
068400,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI
006840,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI
054620,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
347770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
343510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
299030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ


In [3]:
def get_breakpoint(data, breakpoint):
    absolute_point = []
    rank = data.rank(pct=True)
    for i in range(len(breakpoint)):
        absolute_point.append((data[rank < breakpoint[i]].max() + data[rank > breakpoint[i]].min())/2)
    return absolute_point

def get_mask(data, market_cat = None, market_bp = 'All', breakpoint: list = [0.5]):
    # data는 mask의 기준이 되는 baseline data
    # market_bp는 'ALL', 'KOSPI', 'KOSDAQ'으로, 기준이 되는 breakpoint의 market을 의미
    # breakpoint는 masking의 구분점으로, list안에 float이 담긴 형태를 받음
    breakpoint = np.sort(breakpoint)
    mask_list = []
    
    data_ind = data.index
    if type(market_cat) != type(None):
        market_cat = market_cat[market_cat.index.map(lambda x: x in data.index)]
        for i in range(len(data)):
            if data_ind[i] not in market_cat.index:
                market_cat.loc[data_ind[i]] = None
        market_cat = market_cat.reindex(index = data.index)
    for i in range(len(data.columns)):
        mask = data.iloc[:,i] * 0 +1
        mask_np = -np.ones(len(mask))
        #rank = data.iloc[:,i].rank(pct = True)
        if market_bp == 'All':
            absolute_point = get_breakpoint(data.iloc[:,i], breakpoint)
        if market_bp == 'KOSPI':
            absolute_point = get_breakpoint(data[market_cat.iloc[:,i]=='KOSPI'].iloc[:,i], breakpoint)
        if market_bp == 'KOSDAQ':
            absolute_point = get_breakpoint(data[market_cat.iloc[:,i]=='KOSDAQ'].iloc[:,i], breakpoint)
        for j in range(len(breakpoint)+1):
            if j < len(breakpoint):
                for k in range(len(mask)):
                    mask_np[k] = j if data.iloc[k,i] < absolute_point[j] and mask_np[k] == -1 else mask_np[k]
            else:
                for k in range(len(mask)):
                    mask_np[k] = j if data.iloc[k,i] > absolute_point[j-1] and mask_np[k] == -1 else mask_np[k]
        mask = mask * mask_np
        mask_list.append(mask)
    mask_df = pd.concat(mask_list, axis =1)
    return mask_df

In [101]:
get_mask(equity, get_market_category(equity), market_bp = 'KOSPI', breakpoint = [0.2, 0.8])

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,20131231,20140630,20141231,20150630,20151231,20160630,20161231,20170630,20171231,20180630,20181231,20190630,20191231
corp_code,,,,,,,,,,,,,
019660,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
024840,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
067830,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
101170,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0
149950,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
010140,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
096530,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
225190,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def shift_date_quarter(data, num_of_quarters):
    # data의 date를 원하는 만큼 밀어줌.
    dates = data.columns
    years = [int(x[:4]) for x in dates]
    months = [int(x[4:6]) for x in dates]
    shifted_months = [x + num_of_quarters * 3 for x in months]
    shifted_years = [x + (shifted_months[i]-1) // 12 for i, x in enumerate(years)]
    shifted_months = [(x -1) % 12 +1 for x in shifted_months]
    dates = [str(shifted_years[i]) + str(shifted_months[i]).zfill(2)+ '30' if shifted_months[i] in [6, 9] else str(shifted_years[i]) + str(shifted_months[i]).zfill(2)+ '31' for i in np.arange(len(years))]
    data2 = copy.deepcopy(data)
    data2.columns = dates
    return data2

In [105]:
see = get_mask(equity, get_market_category(equity), market_bp = 'KOSPI', breakpoint = [0.3, 0.7])

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [43]:
see.notna()[see.iloc[:,0] == 3].iloc[:,0]

Series([], Name: 20131231, dtype: bool)

In [16]:
get_market_data_on_date('20100104')

,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,069110,3H,KOSDAQ,NaN,1030,10,0.98,1050,1070,1015,39144,40282325,9209333000,8941100
1,060310,3S,KOSDAQ,NaN,1295,120,10.21,1155,1320,1155,1137353,1439119690,38016606635,29356453
2,900010,3노드디지탈,KOSDAQ,NaN,2140,15,0.71,2145,2165,2125,182849,392135450,102720000000,48000000
3,013340,AJS,KOSDAQ,NaN,2840,20,0.71,2815,2895,2795,604360,1715446935,92460690040,32556581
4,054620,AP시스템,KOSDAQ,NaN,4405,130,3.04,4220,4550,4190,745444,3301401335,80497066910,18274022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,000540,흥국화재,KOSPI,NaN,6850,350,5.38,6510,6960,6510,121580,829019700,351008604200,51242132
1957,000547,흥국화재2우B,KOSPI,NaN,5090,290,6.04,5100,5150,5020,1020,5242700,781824000,153600
1958,000545,흥국화재우,KOSPI,NaN,2200,65,3.04,2175,2220,2120,1081,2343700,1689600000,768000
1959,003280,흥아해운,KOSPI,NaN,685,-5,-0.72,685,690,680,134910,92094750,48438699550,70713430


In [70]:
min_return = pd.Series([1,2,3,4,5])[pd.Series([1,2,3,4,5]).rank(pct=True) > 0.3].min()

In [73]:
t[t.rank(pct=True) < 0.3] = min_return

In [74]:
t

0    2
1    2
2    3
3    4
4    5
dtype: int64

In [72]:
t = pd.Series([1,2,3,4,5])

In [76]:
factor = get_factor_on_date_by_mask(see, 'm', weight = 'VW')

NameError: name 'see' is not defined

In [32]:
factor

,0,1,2
201401,inf,inf,0.010395
201402,0.045445,0.052937,0.022845
201403,0.068933,0.034805,0.017769
201404,0.043275,0.032605,-0.003753
201405,0.013479,0.009265,0.003870
201406,-0.011078,-0.011401,0.018490
201407,inf,inf,0.029075
201408,0.046167,0.048035,0.019733
201409,0.018542,0.015562,-0.011917
201410,-0.021380,-0.021901,-0.046235


In [137]:
# 이제 회귀분석 돌린 값을 뽑아내야함
# 어떻게 만들어놓아야 편할까
# 포트폴리오 구축 + 포트폴리오별로 회귀분석
# 문제는 포트폴리오 구축이다. 각 팩터별로 마스킹을 씌워야 하는데 이게 따로 씌우는 게 아니라 사이즈별로 씌워야 한다.
# 기존에 만들어놓은 마스킹 방법론을 사용할 수는 있을 것 같은데..?
# 아 순차적으로 씌우도록 마스킹하면 가능하겠다!


In [6]:
equity = get_item_for_period('ifrs_Equity', '20121231', '20201231')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [7]:
size = get_market_cap_for_period('20121231', '20201231')

In [8]:
size[0]

,20121231,20131231,20141231,20151231,20161231,20171231,20181231,20191231
060310,2.920260e+11,1.815617e+11,9.793698e+10,9.548345e+10,1.168980e+11,7.744228e+10,9.833687e+10,1.262574e+11
900010,5.206450e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
038120,6.412671e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
013340,3.407848e+10,2.371342e+10,NaN,NaN,NaN,NaN,NaN,NaN
068400,2.070679e+11,2.735068e+11,3.609847e+11,2.635410e+11,1.849216e+11,2.194698e+11,2.723995e+11,2.613263e+11
...,...,...,...,...,...,...,...,...
272210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.140885e+12
317320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.157580e+10
340440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.467100e+09
322000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.932000e+11


In [29]:
size2 = get_market_cap_for_period('20130630', '20200630')

# Pfo Construction

In [54]:
# 이거 포트폴리오도 매번 시간대별로 마스킹을 다르게 씌워야함. 반복문 돌려야할 것 같은데..ㅠ
# 그래도 기업별로 돌리는 게 아니라 생각보다는 덜 걸릴 것 같음!
# 우선 가볍게 2 * 3 포트폴리오를 구성해보자
t = pd.DataFrame([[[1,2],[3,4]],[[5,6],[7,8]]])

In [96]:
t.apply(lambda x: x.index, axis=0)

,0,1
0,0,0
1,1,1


In [30]:
size_mask = get_mask(size2[0], size2[1], 'KOSPI', [0.5])

In [31]:
size_mask

,20130630,20140630,20150630,20160630,20170630,20180630,20190630,20200630
060310,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
013340,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
068400,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
006840,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
054620,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
318000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
256150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
272210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
340440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [34]:
get_market_category(equity/size[0])

,20121231,20131231,20141231,20151231,20161231,20171231,20181231,20191231
060310,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ
900010,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
038120,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
013340,KOSDAQ,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN
068400,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI
...,...,...,...,...,...,...,...,...
272210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSPI
317320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
340440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
322000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSPI


In [35]:
hml_mask = get_mask(equity/size[0], get_market_category(equity/size[0]), breakpoint=[0.3,0.7])

In [36]:
hml_mask

,20121231,20131231,20141231,20151231,20161231,20171231,20181231,20191231
000020,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN
000030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000040,1.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN
000050,2.0,2.0,1.0,2.0,2.0,2.0,2.0,NaN
000060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
950160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950170,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
950180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
mask_23 = size_mask * 3 + shift_date_quarter(hml_mask, 2)

In [38]:
mask_23

,20130630,20140630,20150630,20160630,20170630,20180630,20190630,20200630
000020,NaN,NaN,NaN,NaN,4.0,4.0,4.0,NaN
000030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000040,1.0,4.0,0.0,3.0,0.0,1.0,1.0,NaN
000050,5.0,5.0,4.0,5.0,5.0,5.0,5.0,NaN
000060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
950160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950170,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
950180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 각각은 위와 같은 방식으로 구할 수 있다. 
# 순서대로 나누면 어떻게 구해야 하지?
# 함수를 정의할 때 포트폴리오를 구축해주는 함수를 만들어야 할 것 같다. 그래서 쌓으면 자동으로 쌓이도록 만들어야 하지 않을까.


In [8]:
def make_portfolio(pfo, factor, market_bp= 'All', breakpoint=[0.5]):
    max_num = int(pfo.max()[0]+1)
    new_df = pd.DataFrame(index = pfo.index, columns = pfo.columns)
    for i in range(len(pfo.columns)):
        for j in range(max_num):
            get_index = list(filter(lambda x: x in factor.index, pfo.iloc[:,i][pfo.iloc[:,i]==j].index))
            pfo_mask = get_mask(pd.DataFrame(factor.loc[get_index].iloc[:,i]), get_market_category(pd.DataFrame(factor.loc[get_index].iloc[:,i])), market_bp, breakpoint)
            new_df.iloc[:,i].loc[pfo_mask.index] = pfo_mask.values[:,0]
    return new_df

In [40]:
hml_on_size = make_portfolio(size_mask, shift_date_quarter(equity/size[0],2), breakpoint = [0.3,0.7])

In [41]:
hml_on_size.max()

20130630    2.0
20140630    2.0
20150630    2.0
20160630    2.0
20170630    2.0
20180630    2.0
20190630    2.0
20200630    NaN
dtype: float64

In [42]:
size_mask

,20130630,20140630,20150630,20160630,20170630,20180630,20190630,20200630
060310,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
013340,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
068400,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
006840,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
054620,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
318000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
256150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
272210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
340440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [43]:
size_mask.max().max()

1.0

In [44]:
hml_on_size

,20130630,20140630,20150630,20160630,20170630,20180630,20190630,20200630
060310,NaN,0,0,0,0,0,0,NaN
013340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
068400,1,1,1,1,2,2,1,NaN
006840,2,2,1,2,2,2,2,NaN
054620,1,0,1,1,0,2,2,NaN
...,...,...,...,...,...,...,...,...
318000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
def serialize_pfo(mask_list: list):
    # 사전식 배열. 앞에서부터 사전식으로 serialize하여 뱉음
    serial_num = [x.max().max()+1 for x in mask_list]
    for i in reversed(range(len(serial_num)-1)):
        serial_num[i] *= serial_num[i+1]
    serial_num += [1]
    serial_num = serial_num[1:]
    serial_elem = [x * mask_list[i] for i, x in enumerate(serial_num)]
    for i in range(len(serial_elem)-1):
        serial_elem[0] += serial_elem[i+1]
    return serial_elem[0]

In [46]:
size_mask = get_mask(size[0], size[1], breakpoint= [0.2,0.4,0.6,0.8])

In [56]:
shift_date_quarter(size_mask, 2)

,20130630,20140630,20150630,20160630,20170630,20180630,20190630,20200630
060310,4.0,3.0,2.0,2.0,2.0,1.0,2.0,2.0
900010,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
038120,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
013340,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
068400,3.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...
272210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
317320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
340440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
322000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [57]:
serialize_pfo([shift_date_quarter(size_mask, 2), hml_on_size])

,20130630,20140630,20150630,20160630,20170630,20180630,20190630,20200630
060310,NaN,9,6,6,6,3,6,NaN
900010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
038120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
013340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
068400,10,13,13,10,11,11,10,NaN
...,...,...,...,...,...,...,...,...
272210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Regression 돌리기

In [ ]:
# 포트폴리오별 월별 수익률이 나왔으니까 이거 바탕으로 돌려보면 될 듯!
# get factor로 date별 각 factor 구한 다음에 이걸 바탕으로 회귀분석을 돌려야 할 듯!
# 방법론 자체는 monthly, 위에서 추가적으로 작업할 내용으로는 EW인지 VW인지 확실히 알아내야 함.
# -> VW인 것 같은데..? 맞나 뭐 어쨌든 VW EW 둘 다 있어야 함. 선택지로 고려할 수 있도록 만들어야 할 듯.
# 지금은 EW로 만들어놨는데 VW로도 해볼 수 있게 만들어야 할 듯. : get factor

In [29]:
size = get_market_cap_for_period('20141231', '20201231')

In [31]:
SMB = get_mask(size[0], size[1], market_bp = 'KOSPI')

In [32]:
get_factor_on_date_by_mask(SMB, 'm', winsorize_limits = 0.01, weight = 'VW')

KeyboardInterrupt: 

In [24]:
# 우선 돌아가게 하는 데에 집중하고, 속도는 나중에 코드 돌리면서 더빠르게 만들어보자. 
def get_factor_on_date_by_mask(mask, term = 'd', winsorize_limits = 0.01, weight = 'EW'):
    # @@@@@@@@@@@ EW 말고 VW도 추가하면 좋을 듯. 혹은 두 개 combination도 나중에는 고려해봐야할듯.
    # @@@@@@@@@@@ option 추가해야할듯! : daily / weekly / monthly / quarterly 정도까지는 구현을 해놓아야할듯
    # monthly 계산할 때 첫 달은 어떻게 계산하는거지? 전달 말일 기준으로 계산하나? 아니면 해당 달 시초가 기준으로 계산하나?
    # -> 우선 첫 달은 시가 기준, 이후로는 전달 말일에 리밸런싱 하는 걸 기준으로 계산했음. 
    term_conservative = pd.Timestamp(mask.columns[1]) - pd.Timestamp(mask.columns[0]) - pd.Timedelta(days = 7)
    if pd.Timestamp(mask.columns[-1]) + term_conservative > 
    date_to = pd.Timestamp(last_market_date((pd.Timestamp(mask.columns[-1]) + term_conservative).strftime('%Y%m%d')))
    date_from = pd.Timestamp(mask.columns[0])
    print(date_to)
    print(date_to > pd.Timestamp('20210630'))
    if date_to > pd.Timestamp('20210630'):
        print('here')
        date_to = pd.Timestamp('20210630')
    factor_days = (date_to - date_from).days
    factor_months = (date_to.year - date_from.year)*12 + (date_to.month - date_from.month)
    
    date_point = find_next_market_date(date_from)
    # this is for storing the last date
    date_point_marked = None
    
    total_factor = []
    date_list = []
    
    if term == 'd':
        flag = -1
#         for _ in range(factor_days):
# 이거 flag 바뀔 때 date_point_marked 그대로 놔둬도 되나? : 상관 없는듯? 날짜만 저장하는 거라 flag는 어차피 다음 거 기준으로 계산됨.
# 근데 이렇게 되면 없어졌다가 다시 생겨난 기업의 경우 다시 포함되는데 이건 어떻게 처리해야할지 고민좀 해봐야 겠다...
# flag를 따로 저장해서 거기에서 자체적으로 지운 다음에 그걸 레퍼런스 삼아서 계산해야 에러가 없을 듯. 
# winsorize도 추가해야 함.
        for _ in tqdm.tqdm(range(10)):
            if pd.Timestamp(find_next_market_date(date_point)) > pd.Timestamp(mask.columns[flag+1]):
                flag+=1
                c_mask = mask.iloc[:,flag]
            date_point_marked = date_point
            first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '종가', '시가총액']]
            date_point = find_next_market_date(date_point_marked)
            second = get_market_data_on_date(date_point, print_error = False).loc[:,['종목코드', '종가', '시가총액']]
            date_list.append(date_point)
            return_list = [[] for x in np.arange(mask.max()[0]+1)]
            value_list = [[] for x in np.arange(mask.max()[0]+1)]
            for i in range(len(first)):
                if first.iloc[i,0] in c_mask.index and c_mask.notna().loc[first.iloc[i,0]]:
                    if second.notna()[second['종목코드'] == first.iloc[i,0]].iloc[0,0]:
                        return_list[int(c_mask.loc[first.iloc[i,0]])].append((second[second['종목코드'] == first.iloc[i,0]].iloc[0,1] - first.iloc[i,1])/first.iloc[i,1])
                    else:
                        return_list[int(c_mask.loc[first.iloc[i,0]])].append(-0.99)
                    value_list[int(c_mask.loc[first.iloc[i,0]])].append(first.iloc[i,2])
                    
            print(np.mean(return_list[0]))
            if weight == 'VW':
                for i in range(len(return_list)):
                    # len을 곱해준 건 나중에 mean에서 EW와 같은 형태로 들어가게 하도록 하기 위함.
                    return_list[i] *= value_list[i]/ np.sum(value_list[i]) * len(value_list[i])
                    
            print(np.mean(return_list[0]))
            # winsorize
            if winsorize_limits > 0:
                for i in range(len(return_list)):
                    return_list[i] = winsorize(return_list[i], limits = winsorize_limits)
            print(np.mean(return_list[0]))
                
            factor = [np.mean(x) for x in return_list]
            total_factor.append(factor)
        return pd.DataFrame(total_factor, index = date_list)
    elif term == 'm':
        flag = -1
        for _ in tqdm.tqdm(range(factor_months)):
            if flag +1 == mask.shape[1]:
                c_mask = mask.iloc[:,flag]
                # 기준이 바뀌었을 경우에는 기간 시초가를 기준으로 계산
                date_point = find_next_market_date(date_point)
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '시가', '시가총액']]
            elif pd.Timestamp(find_next_market_date(date_point)) > pd.Timestamp(mask.columns[flag+1]):
                flag+=1
                c_mask = mask.iloc[:,flag]
                # 기준이 바뀌었을 경우에는 기간 시초가를 기준으로 계산
                date_point = find_next_market_date(date_point)
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '시가', '시가총액']]
            else:
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '종가', '시가총액']]
            
            # 통일된 수식으로 flag가 바뀐 경우의 말일을 계산할 수 있음
            date_point = last_market_date(find_next_market_date(date_point_marked))
            second = get_market_data_on_date(date_point, print_error = False).loc[:,['종목코드', '종가', '시가총액']]
            date_list.append(date_point[:6])
            return_list = [[] for x in np.arange(mask.max()[0]+1)]
            value_list = [[] for x in np.arange(mask.max()[0]+1)]
            for i in range(len(first)):
                if first.iloc[i,0] in c_mask.index and c_mask.notna().loc[first.iloc[i,0]]:
                    if len(second.notna()[second['종목코드'] == first.iloc[i,0]]) > 0:
                        if second.notna()[second['종목코드'] == first.iloc[i,0]].iloc[0,0]:
                            if first.iloc[i,1] == 0:
                                return_list[int(c_mask.loc[first.iloc[i,0]])].append(0)
                            else:
                                return_list[int(c_mask.loc[first.iloc[i,0]])].append((second[second['종목코드'] == first.iloc[i,0]].iloc[0,1] - first.iloc[i,1])/first.iloc[i,1])
                        else:
                            return_list[int(c_mask.loc[first.iloc[i,0]])].append(-0.99)
                    else:
                        return_list[int(c_mask.loc[first.iloc[i,0]])].append(-0.99)
                    value_list[int(c_mask.loc[first.iloc[i,0]])].append(first.iloc[i,2])
            if weight == 'VW':
                for i in range(len(return_list)):
                    # len을 곱해준 건 나중에 mean에서 EW와 같은 형태로 들어가게 하도록 하기 위함.
                    return_list[i] *= value_list[i]/ np.sum(value_list[i]) * len(value_list[i])
            # winsorize
            if winsorize_limits > 0:
                for i in range(len(return_list)):
                    return_list[i] = winsorize(return_list[i], limits = winsorize_limits)
            factor = [np.mean(x) for x in return_list]
            total_factor.append(factor)
        return pd.DataFrame(total_factor, index = date_list)